# How to publish a web service to Azure ML using R

## Introduction
The purpose of this notebook is to demonstrate how to use Jupyter notebooks on the Azure Machine Learning (ML) platform to develope a model in R and publish a web service based on the model.

### Why Azure ML R notebooks
The answer to this question depends on how much you know about Azure ML. For data scientists who're new to Azure ML and accustomed to doing all analytical work using R on local computers,  Azure Machine Learning makes it possible to write R notebooks on the cloud. So anyone with internet access can work with R from a web browser. 

If you use R and understand the basics of Azure ML, Azure ML's R notebooks make it possible to develope your models in R and then operationalize them easily. 

For data scientists who're comfortable with both R and Azure ML Experiments, R notebooks can be used together with Azure ML Experiments in different ways:
  * To explore data from Azure Machine Learning Experiment. For example, you can use R notebook to visualize your data in different ways.
  * To fit models and use techniques that are not available in Azure ML Experiments yet. For example, R offers more options in terms of variable selection techniques and a wider variety of GBM models. You can also used it for time series analysis.
  * To test code before they are used in the "Execute R Script" module of Azure ML Experiments.

### Target audience
The target audience of this notebook are R users who have a basic understanding of Azure ML. If you are new to Azure ML, Section 2 of the [Data Scientists' Guide][guide link] provides enough information for you to follow this tutorial. 

[guide link]: https://gallery.cortanaanalytics.com/Experiment/Tutorial-for-Data-Scientists-3


## Data
In this example, we'll use the housing data from the R package `MASS`.  The Boston data contains 506 rows and 14 columns. Available information includes median home price, average number of rooms per dwelling, crime rate by town, etc. Find more information about this dataset by running `?Boston` in an R terminal.

In [ ]:
?Boston

## A linear regression model
For illustration purposes, use `medv` - median home price - as the response variable and the remaining variables as predictors.

[uci link]: https://archive.ics.uci.edu/ml/datasets/Housing

In [ ]:
# load the library to use the Boston dataset
library(MASS)

# fit a model using all variables except medv as predictors
lm1 <- lm(medv ~ ., data = Boston)

# check model performance
summary(lm1)

In [ ]:
pred <- predict(lm1)
error <- pred-Boston$medv
mae <- mean(abs(error))
rmse <- sqrt(mean((error)^2))
rae <- mean(abs(error))/mean(abs(Boston$medv-mean(Boston$medv)))
rse <- mean((error)^2)/mean((Boston$medv-mean(Boston$medv))^2)

cat("Mean Absolute Error:", round(mae, 6), "\n")
cat("Mean Absolute Error:", round(mae, 6), "\n")
cat("Root Mean Squared Error:", round(rmse, 6), "\n")
cat("Relative Absolute Error:", round(rae, 6), "\n")
cat("Relative Squared Error:", round(rse, 6), "\n")

## Web service
### Deploy a web service
With the developed model, you can deploy a web service so that others can use it to make predictions. The `AzureML` R package will be used for this purpose. You'll need to provide the **workspace ID** and **authorization token** for an Azure ML workspace. The two screenshots below show where you can find them in your workspace. Either primary or secondary authorization token can be used. 

#### Finding your workspace ID

<img src="https://cloud.githubusercontent.com/assets/9322661/11348789/952a5db4-91f6-11e5-9710-8805451194dd.PNG" width = "800", alt = "Where to find workspace ID">

#### Finding your authorization token

<img src="https://cloud.githubusercontent.com/assets/9322661/11348692/09746e40-91f6-11e5-9dfa-6ac897e3c426.PNG" width = "800">



The code below sets up a web service. 

In [ ]:
# load the library
library(AzureML)

# retrieve workspace information
ws <- workspace()

# define predict function
mypredict <- function(newdata){
  predict(lm1, newdata)
}

# a sample with predictor information
newdata <- Boston[1:5, ]

# test the prediction function
data.frame(actual = newdata$medv, prediction = mypredict(newdata))

In [ ]:
# publish the service
ep <- publishWebService(ws = ws, 
                        fun = mypredict, 
                        name = "HousePricePrediction", 
                        inputSchema = newdata)
str(ep)

### Consume a web service
After setting up a web service, you can use R scripts to consume it in three ways: one is in-session and two are out-of-session.

#### In-session consumption
If you are consuming the web service in the same session that the web service was set up, you can refer to the endpoint directly.

In [ ]:
pred <- consume(ep, newdata)$ans
data.frame(actual = newdata$medv, prediction = pred)

# Note that this operation may take several seconds to complete.
# You may see several retry attempts, before the service returns with a value

#### Out-of-session consumption
If you consume the web service in a new session, you can do it in two ways. In the first approach, you need to save the workspace information - workspace id and  authorization token - and web service ID. Such information can then be used by the consume\(\) function as shown below.

In [ ]:
# Create a temporary file to store the endpoint data
tf <- tempfile(fileext = ".rds")
saveRDS(ep, file = tf)

# Read the endpoint data from file
endpoint <- readRDS(tf)

# consume
pred <- consume(endpoint, newdata)$ans
data.frame(actual = newdata$medv, prediction = mypredict(newdata))

Alternatively, you can save the API information and call it directly. 

### Update a web service
After making improvements to a model, you can update the existing web service. For this purpose you can use the updateWebService\(\) function by specifying the web service ID. 

In [ ]:
# define test function
mypredictnew <- function(newdata){
  predict(lm1, newdata) + 100
}

# update service with the new function
ep_update <- updateWebService(
  ws = ws,
  fun = mypredictnew, 
  name = "newpredict", # this does not matter since serviceId is provided
  inputSchema = newdata, 
  serviceId = service_id   
)

## Conclusion
Through this example, you've learned how to fit a model, deploy the model on Azure, and consume the service, all using Azure ML's Jupyter notebook.

The `AzureML`R package also allows you to read data from Azure ML workspace or experiments, making it possible for users to have a seamless experience between Azure ML experiments and notebooks. Details can be found in the package documentation.

---  
Created by a Microsoft Employee.  
Copyright (C) Microsoft. All Rights Reserved.